# MLP with Adam Using Tensorflow - Tuning

In [1]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import wandb
from pprint import pprint

from tf_utils import create_model, train_model

## Login to Wandb

In [2]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Load Fashion-MNIST

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

4422102/4422102 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


## Define sweep agent function

In [4]:
def sweep_train():
    default_config = {
        'dense_1': 512,
        'dense_2': 256,
        'dense_3': 256,
        'dense_4': 64,
        'learning_rate': 0.001,
        'beta_1': 0.9,
        'beta_2': 0.999,
        'epochs': 50,
        'batch_size': 128,
    }

    wandb.init(config=default_config)

    config = wandb.config

    mlp = create_model(config)
    mlp.summary()
    mlp.compile(
        optimizer=Adam(learning_rate=config.learning_rate, beta_1=config.beta_1, beta_2=config.beta_2),
        loss=SparseCategoricalCrossentropy(),
        metrics=[SparseCategoricalAccuracy()]
    )
    _ = train_model(mlp, config, x_train, y_train, verbose=0)

## Configure Wandb Sweep for hyperparameter tuning

In [5]:
sweep_config = {
    'name': 'tf-adam-sweep',
    'method': 'bayes',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'early_terminate':{
        'type': 'hyperband',
        'min_iter': 2
    },
    'parameters': {
        'batch_size': {
            'values': [16, 32, 64, 128, 256]
        },
        'epochs': {
            'min': 15,
            'max': 50
        },
        'learning_rate': {
            'min': 0.0001,
            'max': 0.1
        },
        'beta_1': {
            'min': 0.9,
            'max': 0.999
        },
        'beta_2': {
            'min': 0.99,
            'max': 0.9999
        },
        'dense_1': {
            'min': 256,
            'max': 512
        },
        'dense_2': {
            'min': 128,
            'max': 256
        },
        'dense_3': {
            'min': 128,
            'max': 256
        },
        'dense_4': {
            'min': 32,
            'max': 128
        }
    }
}

pprint(sweep_config)

sweep_id = wandb.sweep(sweep_config, project='tf_mlp')

{'early_terminate': {'min_iter': 2, 'type': 'hyperband'},
 'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'name': 'tf-adam-sweep',
 'parameters': {'batch_size': {'values': [16, 32, 64, 128, 256]},
                'beta_1': {'max': 0.999, 'min': 0.9},
                'beta_2': {'max': 0.9999, 'min': 0.99},
                'dense_1': {'max': 512, 'min': 256},
                'dense_2': {'max': 256, 'min': 128},
                'dense_3': {'max': 256, 'min': 128},
                'dense_4': {'max': 128, 'min': 32},
                'epochs': {'max': 50, 'min': 15},
                'learning_rate': {'max': 0.1, 'min': 0.0001}}}
Create sweep with ID: 12a25071
Sweep URL: https://wandb.ai/nsiete23/tf_mlp/sweeps/12a25071


## Run sweep

In [6]:
wandb.agent(sweep_id, function=sweep_train, count=50)

wandb: Agent Starting Run: qdcc9m2a with config:
wandb: 	batch_size: 32
wandb: 	beta_1: 0.9055461392449808
wandb: 	beta_2: 0.9955683606550336
wandb: 	dense_1: 502
wandb: 	dense_2: 150
wandb: 	dense_3: 227
wandb: 	dense_4: 123
wandb: 	epochs: 48
wandb: 	learning_rate: 0.09416619525165897
wandb: Currently logged in as: v-modroczky (nsiete23). Use `wandb login --relogin` to force relogin


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 28, 28)            0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 502)               394070    
                                                                 
 dense_1 (Dense)             (None, 150)               75450     
                                                                 
 dense_2 (Dense)             (None, 227)               34277     
                                                                 
 dense_3 (Dense)             (None, 123)               28044     
                                                                 
 dense_4 (Dense)             (None, 10)                1

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sparse_categorical_accuracy,▄▃▃▂▅▃▅▅▃█▄▃▄▄▂▃▄▅▁▂▂▃▆▁▃▂▇▃▅▇▃▄▄▅▆▅▃▄▃▄
val_loss,▂▄█▃▇▆▆▁▂▂▄▃▂▁▄▃▁▄▃▁▃▄▃▃▃▃▂▆▂▂▃▃▃▃▂▂▃▃▅▃
val_sparse_categorical_accuracy,▅▃▁███▅▃▆█▅▅▅▅▃█▅█▅█▃▃▅▃▃▃▁▅█▃▅█▆▆██▃▅█▅
GFLOPS,0.00053
best_epoch,35
best_val_loss,2.30686
epoch,47
loss,2.31525
sparse_categorical_accuracy,0.10027


wandb: Agent Starting Run: nnzsrrxb with config:
wandb: 	batch_size: 256
wandb: 	beta_1: 0.941964814245118
wandb: 	beta_2: 0.994023947453032
wandb: 	dense_1: 305
wandb: 	dense_2: 183
wandb: 	dense_3: 206
wandb: 	dense_4: 58
wandb: 	epochs: 32
wandb: 	learning_rate: 0.03703548823282531


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 28, 28)            0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 305)               239425    
                                                                 
 dense_1 (Dense)             (None, 183)               55998     
                                                                 
 dense_2 (Dense)             (None, 206)               37904     
                                                                 
 dense_3 (Dense)             (None, 58)                12006     
                                                                 
 dense_4 (Dense)             (None, 10)                5

...

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 28, 28)            0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 273)               214305    
                                                                 
 dense_1 (Dense)             (None, 151)               41374     
                                                                 
 dense_2 (Dense)             (None, 200)               30400     
                                                                 
 dense_3 (Dense)             (None, 87)                17487     
                                                                 
 dense_4 (Dense)             (None, 10)                8

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
sparse_categorical_accuracy,▁▅▆▆▆▆▇▇▇▇▇▇▇▇███████████
val_loss,█▃▂▂▄▄▂▂▁▂▁▁▂▁▂▃▂▁▃▁▃▃▂▆▄
val_sparse_categorical_accuracy,▁▅▅▆▅▅▆▆▆▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇
GFLOPS,0.0003
best_epoch,13
best_val_loss,0.35723
epoch,24
loss,0.25335
sparse_categorical_accuracy,0.90977


wandb: Agent Starting Run: vo7fu6ix with config:
wandb: 	batch_size: 256
wandb: 	beta_1: 0.9888964593164976
wandb: 	beta_2: 0.992148909221722
wandb: 	dense_1: 272
wandb: 	dense_2: 248
wandb: 	dense_3: 234
wandb: 	dense_4: 125
wandb: 	epochs: 29
wandb: 	learning_rate: 0.022947780943994897


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 28, 28)            0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 272)               213520    
                                                                 
 dense_1 (Dense)             (None, 248)               67704     
                                                                 
 dense_2 (Dense)             (None, 234)               58266     
                                                                 
 dense_3 (Dense)             (None, 125)               29375     
                                                                 
 dense_4 (Dense)             (None, 10)                1

epoch,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇██
loss,█▃▂▂▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁
sparse_categorical_accuracy,▁▅▆▇▇▇▇▇▇▇█████▇███▇▇▇███████
val_loss,█▇▃▃▁▂▂▂▁▂▃▁▃▁▂▂▃▃▃▃▆▃▂▅▂▂▂▃▂
val_sparse_categorical_accuracy,▁▃▅▅▇▇██▇▇▅▇▆█▇▇▇▇▇▆▆▆█▇██▇▇▇
GFLOPS,0.00037
best_epoch,4
best_val_loss,0.44566
epoch,28
loss,0.39973
sparse_categorical_accuracy,0.8644


## Show best hyperparameters

In [7]:
api = wandb.Api()
sweep = api.sweep("nsiete23/tf_mlp/sweeps/" + sweep_id)

best_run = sweep.best_run()
print(best_run.id)
pprint(best_run.config)

wandb: Sorting runs by +summary_metrics.val_loss


40dunp3m
{'batch_size': 128,
 'beta_1': 0.986435025614692,
 'beta_2': 0.9904482393527106,
 'dense_1': 485,
 'dense_2': 176,
 'dense_3': 131,
 'dense_4': 122,
 'epochs': 19,
 'learning_rate': 0.00012885226472854095}


## Finish Wandb run

In [8]:
wandb.finish()